# Project 3 — Titanic Classification
**Name:** Brandon  
**Goal:** Compare DT, SVC, NN on 3 cases (alone | age | age+family_size) with stratified split.


In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score

plt.rcParams['figure.figsize'] = (8,6)



ModuleNotFoundError: No module named 'seaborn'

## Step 2 — Load and Inspect the Data
We’ll use Seaborn’s built-in Titanic dataset to preview and understand what columns we can work with.


In [ ]:
import seaborn as sns
titanic = sns.load_dataset('titanic')
titanic.head()


In [ ]:
titanic.info()
titanic.describe(include='all')


## Step 3 — Clean & Feature Engineer
- Impute `age` (median) and `embark_town` (mode)
- Create `family_size = sibsp + parch + 1`
- Convert `alone` to int (0/1) for modeling


In [ ]:
# Impute missing values
titanic['age'] = titanic['age'].fillna(titanic['age'].median())
titanic['embark_town'] = titanic['embark_town'].fillna(titanic['embark_town'].mode()[0])

# Feature engineering
titanic['family_size'] = titanic['sibsp'] + titanic['parch'] + 1
titanic['alone'] = titanic['alone'].astype(int)

# Quick sanity checks
print("Null rates (top 10):")
print(titanic.isna().mean().sort_values(ascending=False).head(10))

titanic[['survived','age','embark_town','sibsp','parch','family_size','alone']].head()


: 